In [2]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np  # Import numpy for numerical operations
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Import StandardScaler for feature scaling and LabelEncoder for encoding target labels
from xgboost import XGBClassifier  # Import XGBClassifier for XGBoost model
from lightgbm import LGBMClassifier  # Import LGBMClassifier for LightGBM model
from catboost import CatBoostClassifier  # Import CatBoostClassifier for CatBoost model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score  # Import specific metrics for model evaluation
import warnings  # Import warnings to manage warnings
import category_encoders as ce  # Import category_encoders for encoding categorical features
import optuna
# Ignore warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [3]:
path = r'C:\Users\User\Desktop\Rashad\DATA\generated data\customer_churn_data.csv'

# Load dataset from Excel file
df = pd.read_csv(path)

# Set option to display all columns
pd.set_option('display.max_columns', None)

#Display the loaded dataset
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,CUST0000,Male,0,No,Yes,23,No,No phone service,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer,49.85,1146.55,No
1,CUST0001,Female,0,Yes,No,43,No,No phone service,DSL,Yes,No,Yes,No,Yes,No,Month-to-month,No,Mailed check,100.70,4330.10,Yes
2,CUST0002,Male,1,No,No,51,Yes,No,DSL,No,Yes,Yes,Yes,No,No,One year,No,Electronic check,97.33,4963.83,Yes
3,CUST0003,Male,1,No,No,72,Yes,Yes,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Credit card,101.38,7299.36,No
4,CUST0004,Male,1,No,No,25,Yes,Yes,DSL,No,No,No,Yes,No,Yes,Month-to-month,No,Electronic check,52.22,1305.50,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,CUST5875,Male,0,Yes,Yes,71,Yes,No,DSL,Yes,No,No,Yes,No,No,Month-to-month,Yes,Mailed check,74.21,5268.91,No
5876,CUST5876,Male,0,No,No,22,Yes,Yes,DSL,No,No,Yes,Yes,No,Yes,One year,No,Mailed check,65.43,1439.46,Yes
5877,CUST5877,Female,0,No,No,68,No,No phone service,Fiber optic,Yes,No,Yes,Yes,Yes,No,One year,Yes,Bank transfer,59.78,4065.04,No
5878,CUST5878,Female,0,Yes,Yes,14,No,No phone service,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Month-to-month,Yes,Mailed check,91.88,1286.32,No


In [4]:
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

In [6]:
df.select_dtypes('object').columns

Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [7]:
models = ["xgboost", "lightgbm", "catboost", "catboost_custom"]

# Create copies of the DataFrame for different preprocessing approaches
df_boost = df.copy()
df_cat = df.copy()

# Iterate through each model in the list
for model in models:            
    if model in ["xgboost", "lightgbm", "catboost"]:
        # Preprocessing for tree-based models
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_boost.columns.tolist()
        all_columns.remove('Churn')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_boost[all_columns] = encoder.fit_transform(df_boost[all_columns], df_boost['Churn'])
             
    elif model ==  "catboost_custom":
        # Preprocessing for CatBoost with custom handling of missing values
        
        # Fill missing values in specified categorical columns with 'Missing Value'
        numerical_cols = df_cat.select_dtypes(include=['int64', 'float64']).columns.tolist()
        numerical_cols.remove('Churn')
        columns_to_fill = cat_features=['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod']
        df_cat[columns_to_fill] = df_cat[columns_to_fill].fillna('Missing Value')
        
        # Perform Weight of Evidence Encoding on numerical columns
        cat_encoder = ce.WOEEncoder(cols=numerical_cols)
        df_cat[numerical_cols] = cat_encoder.fit_transform(df_cat[numerical_cols], df_cat['Churn'])

# Split data into features (X) and target (y) for each preprocessing approach
x_boost = df_boost.drop(columns=['Churn'])
y_boost = df_boost['Churn']
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(x_boost, y_boost, test_size=0.3, random_state=42)

x_cat = df_cat.drop(columns=['Churn'])
y_cat = df_cat['Churn']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(x_cat, y_cat, test_size=0.3, random_state=42)

In [8]:
models = []

xgb_model_def = XGBClassifier()
lgb_model_def = LGBMClassifier()
catboost_model_def = CatBoostClassifier()
catboost_model_custom = CatBoostClassifier(cat_features=['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'])

models.extend([
    ('XGBoost', xgb_model_def),
    ('LightGBM', lgb_model_def),
    ('CatBoost', catboost_model_def),
    ('CatBoost_Custom', catboost_model_custom),
])

In [9]:
def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the given model on the training and testing data.

    Parameters:
    model_name (str): Name of the model for display purposes.
    model : Machine learning model object.
    X_train : Features of the training data.
    y_train : Target labels of the training data.
    X_test : Features of the testing data.
    y_test : Target labels of the testing data.

    Returns:
    float, float: Gini coefficients calculated from the model's predictions on training and testing data.
    """

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict labels and probabilities on the testing data
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1]

    # Predict labels and probabilities on the training data
    y_train_pred = model.predict(X_train)
    y_train_prob = model.predict_proba(X_train)[:, 1]

    # Calculate ROC AUC and Gini coefficient for testing data
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1

    # Calculate ROC AUC and Gini coefficient for training data
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1

    # Calculate confusion matrix and classification report for testing data
    confusion_matrix_test_result = confusion_matrix(y_test, y_test_pred)
    classification_report_test_result = classification_report(y_test, y_test_pred)

    # Calculate confusion matrix and classification report for training data
    confusion_matrix_train_result = confusion_matrix(y_train, y_train_pred)
    classification_report_train_result = classification_report(y_train, y_train_pred)

    # Print model performance metrics
    print(f'Model Performance for {model_name}')
    print('Gini prob for testing data is', gini_test_prob * 100)
    print('Gini prob for training data is', gini_train_prob * 100)
    print('Classification Report for Testing Data:')
    print(classification_report_test_result)
    print('Confusion Matrix for Testing Data:')
    print(confusion_matrix_test_result)
    print('Classification Report for Training Data:')
    print(classification_report_train_result)
    print('Confusion Matrix for Training Data:')
    print(confusion_matrix_train_result)

    return gini_train_prob, gini_test_prob

In [12]:
# Initialize the DataFrame to store Gini coefficients
gini_df = pd.DataFrame(columns=['Model', 'Gini_train_prob', 'Gini_test_prob'])

# Iterate through each model in the list of models
for model_name, model in models:
    # Train and evaluate the model, and calculate the Gini coefficient
    if model_name == 'CatBoost_Custom':
        gini_train_prob, gini_test_prob = train_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
    else:
        gini_train_prob, gini_test_prob = train_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
    
    # Add model name and Gini coefficients to the DataFrame
    gini_df = pd.concat([gini_df, pd.DataFrame({'Model': [model_name], 'Gini_train_prob': [gini_train_prob], 'Gini_test_prob': [gini_test_prob]})], ignore_index=True)

# Sort the DataFrame by Gini coefficient for testing data in descending order
gini_df_sorted = gini_df.sort_values(by='Gini_test_prob', ascending=False)


Model Performance for XGBoost
Gini prob for testing data is 36.782840099760875
Gini prob for training data is 97.84590291585083
Classification Report for Testing Data:
              precision    recall  f1-score   support

           0       0.58      0.64      0.61       874
           1       0.61      0.55      0.58       890

    accuracy                           0.60      1764
   macro avg       0.60      0.60      0.60      1764
weighted avg       0.60      0.60      0.60      1764

Confusion Matrix for Testing Data:
[[561 313]
 [400 490]]
Classification Report for Training Data:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2108
           1       0.95      0.93      0.94      2008

    accuracy                           0.94      4116
   macro avg       0.94      0.94      0.94      4116
weighted avg       0.94      0.94      0.94      4116

Confusion Matrix for Training Data:
[[2003  105]
 [ 134 1874]]
[LightGBM] [Info

In [13]:
gini_df_sorted

,Model,Gini_train_prob,Gini_test_prob
3,CatBoost_Custom,0.509492,0.441313
2,CatBoost,0.869166,0.409799
1,LightGBM,0.891582,0.375088
0,XGBoost,0.978459,0.367828
